In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer
import time
from sklearn.preprocessing import LabelEncoder

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [59]:
df_with_seasonal = pd.read_csv("train_with_seasonal.csv")
df_no_seasonal = pd.read_csv("train_filtered2.csv")

In [60]:
df_no_seasonal = df_no_seasonal.drop(columns=["date"])

In [61]:
target_col = "electricity_consumption"

In [62]:
def train_and_eval(df_input, label):
    X = df_input.drop(columns=[target_col])
    y = df_input[target_col]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    model = XGBRegressor(objective="reg:squarederror", n_estimators=300, learning_rate=0.1, max_depth=5)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"✅ {label} RMSE: {rmse:.4f}")
    return rmse

In [63]:
df_no_seasonal = df.drop(columns=["date", "ID", "cluster_id"])
print("🔍 Evaluating without seasonal features...")
rmse_plain = train_and_eval(df_no_seasonal, "Without seasonal features")

🔍 Evaluating without seasonal features...
✅ Without seasonal features RMSE: 245.5917


In [64]:
df_with_seasonal.head()

,ID,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_direction_10m_dominant,month,dayofweek,quarter,day,weekofyear
0,cluster_1_2014-01-01,cluster_1,358.032,10.8,4.2,53.003333,29787.533333,40.6,186.000000,1,2,1,1,1
1,cluster_2_2014-01-01,cluster_2,548.247,12.2,4.3,8195.656667,30650.350000,36.0,170.591118,1,2,1,1,1
2,cluster_3_2014-01-01,cluster_3,758.303,12.9,-0.8,16305.260000,31547.686667,20.9,159.467752,1,2,1,1,1
3,cluster_4_2014-01-01,cluster_4,1072.077,10.8,4.7,9224.803333,30769.220000,34.7,184.339753,1,2,1,1,1
4,cluster_1_2014-01-02,cluster_1,386.908,10.7,7.0,22372.000000,29850.226667,34.3,210.333465,1,3,1,2,1


In [65]:
df_with_seasonal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11688 entries, 0 to 11687
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           11688 non-null  object 
 1   cluster_id                   11688 non-null  object 
 2   electricity_consumption      11688 non-null  float64
 3   temperature_2m_max           11688 non-null  float64
 4   temperature_2m_min           11688 non-null  float64
 5   sunshine_duration            11688 non-null  float64
 6   daylight_duration            11688 non-null  float64
 7   wind_speed_10m_max           11688 non-null  float64
 8   wind_direction_10m_dominant  11688 non-null  float64
 9   month                        11688 non-null  int64  
 10  dayofweek                    11688 non-null  int64  
 11  quarter                      11688 non-null  int64  
 12  day                          11688 non-null  int64  
 13  weekofyear      

In [66]:
df_with_seasonal = df_with_seasonal.drop(columns=["ID", "cluster_id"])
print("\n🔁 Evaluating with seasonal features...")
rmse_seasonal = train_and_eval(df_with_seasonal, "With seasonal features")


🔁 Evaluating with seasonal features...
✅ With seasonal features RMSE: 220.7772
